<a href='http://www.holoviews.org'><img src="assets/hv+bk.png" alt="HV+BK logos" width="40%;" align="left"/></a>
<div style="float:right;"><h2>Excercise 3: Networks and GeoViews</h2></div>

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import geoviews as gv
import networkx as nx

from bokeh.sampledata.airport_routes import airports, routes
from holoviews.operation.datashader import datashade, directly_connect_edges, bundle_graph

hv.extension('bokeh')

In [ ]:
airports
airports.head()

### Example 1

In this example we will plot the coordinates of all of these airports on a map using ``gv.Points``.

In [ ]:
# Create a gv.Points object displaying the 'Latitude' and 'Longitude' dimensions
points = gv.Points(airports, ['Longitude', 'Latitude'])
points

In [ ]:
points = points.select(Longitude=(-180, 0), Latitude=(0, None))
points

Now set the ``width`` to 500 and ``height`` to 400 using ``%%opts`` or the ``.opts`` method.

Now add a gv.WMTS element below the points.

In [ ]:
%%opts Points [width=500 height=400 tools=['hover']] (size=2 color='black' fill_alpha=0)
url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}.png'
gv.WMTS(url) * points

## Exercise 2

In [ ]:
airports.head()

In [ ]:
routes.head()

In [ ]:
%%opts Graph (node_size=4)
g = nx.from_pandas_dataframe(routes, 'SourceID', 'DestinationID')
hv.Graph.from_networkx(g, nx.spring_layout)

In [ ]:
%%opts Graph [width=600 height=600 color_index='TZ'] (node_size=4)
g = nx.from_pandas_dataframe(routes, 'SourceID', 'DestinationID')
ds = hv.Dataset(airports, ['AirportID'], ['Name', 'TZ'])
hv.Graph.from_networkx(g, nx.spring_layout, ds)

In [ ]:
g = nx.from_pandas_dataframe(routes, 'SourceID', 'DestinationID')
ds = hv.Dataset(airports, ['AirportID'], ['Name', 'TZ'])
graph = hv.Graph.from_networkx(g, nx.spring_layout, ds)

opts = {'plot': dict(width=500, height=500, xaxis=None, yaxis=None, color_index='TZ', show_legend=True),
        'style': dict(cmap='tab20', node_size=5, edge_line_width=1)}
graph.opts(**opts)

### Exercise 3

In [ ]:
# Count the number of connections from each airport
counts = routes.groupby('SourceID')[['Stops']].count().reset_index().rename(columns={'Stops': 'Connections'})
airports_df = pd.merge(airports, counts, left_on='AirportID', right_on='SourceID', how='left')

# Select only US mainland airports & convert from Mercator to Latitudes/Longitudes
airport_points = gv.Points(airports_df, ['Longitude', 'Latitude'])
projected_points = gv.operation.project_points(airport_points)

busiest_ids = list(routes.groupby('SourceID').count().sort_values('Stops').iloc[-50:].index.values)
tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')

In [ ]:
# Declare nodes, graph and tiles
nodes = hv.Nodes(projected_points, ['Longitude', 'Latitude', 'AirportID'], ['Name', 'City', 'Connections'])
graph = hv.Graph((routes, nodes), ['SourceID', 'DestinationID'], ['Source', 'Destination'])

# Select 50 busiest airports
busiest_airports = graph.select(AirportID=busiest, selection_mode='nodes')

In [ ]:
%%opts Graph [width=800 height=800] (edge_selection_line_color='black' edge_hover_line_color='red')
%%opts Graph (node_size=8 edge_line_width=1 edge_line_alpha=0 edge_nonselection_line_alpha=0)
tiles * busiest_airports